In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams
from matplotlib import cm, colors, _cm
rcParams['font.family'] = 'NanumGothic'
import string

In [2]:
%matplotlib inline

In [3]:
#데이터 수집
import urllib
import json
from time import time
import sqlite3

In [4]:
state_info = pd.DataFrame(dict(si_code = (11,12), name = ("서울특별시","부산광역시")))

In [5]:
state_info

,si_code,name
0,11,서울특별시
1,12,부산광역시


In [6]:
# si, gun, gu data 수집 
def getAddrInfo( code, type='SGG'  ):
    response = urllib.request.urlopen("http://www.k-apt.go.kr/cmmn/bjd/getBjdList.do?bjd_code=%d&bjd_gbn=%s" %(code,type) )
    data = json.loads(response.read().decode('utf-8'))
    return pd.DataFrame(data[u'resultList'])
# apt kapt code 정보 수집 
# http://www.k-apt.go.kr/kaptinfo/getKaptList.do?bjd_code=11350105
def getAptInfo(code):
    response = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptList.do?bjd_code=%d" % (code) )
    data = json.loads(response.read().decode('utf-8'))
    return pd.DataFrame(data[u'resultList'])

In [7]:
state_info[['name','si_code']].itertuples(index=False)

In [9]:
code = 11
gu_info = getAddrInfo(code)
values=gu_info.apply(lambda x : u"(%d, %s, '%s')" % (code, x['CODE'], x['CODE_VALUE'] ), axis=1)
#insert_sql = u"insert into addr_gu_code_tbl( si_code, gu_code, name) values %s ;" % (u",".join(values))
#con.execute(insert_sql)
#con.commit()

In [10]:
code = (11,680)
si = 11; gu = 680

In [11]:
code = "%d%d" % (si,gu)
info = getAddrInfo(int(code),'EMD')
values=info.apply(lambda x : u"(%d, %d, %s, '%s')" % (si,gu, x['CODE'], x['CODE_VALUE'] ), axis=1)

In [12]:
dong_info = pd.DataFrame(dict(dat = values.str.translate(str.maketrans("","",string.punctuation)).str.split()))

In [13]:
dong_info = dong_info.dat.apply(pd.Series,index = ["si_code","gu_code","dong_code","name"])

In [14]:
dong_info = dong_info.astype(dict(si_code = int, gu_code = int, dong_code = int))

In [15]:
# http://www.k-apt.go.kr/kaptinfo/getKaptList.do?bjd_code=11350105
def getAptDetailInfo(code):
    response = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=%s" % (code) )
    data = json.loads(response.read().decode('utf-8'))
    return data
def extractAptInfo(a):
    info = []
    for name in ['KAPT_CODE','CONVENIENT_FACILITY','EDUCATION_FACILITY','KAPTD_PCNT','KAPTD_PCNTU', 'KAPTD_WTIMEBUS', 'KAPTD_WTIMESUB','SUBWAY_LINE','SUBWAY_STATION' ] :
        info.append(a[u'resultMap_kapt'][name]);
    cnt = 0
    for x in a[u'resultMap_kapt_areacnt']:
        if( type(x[u'KAPTDA_CNT']) is int ):
            info.append(x[u'KAPTDA_CNT'])
        else :
            info.append(0)
#           cnt = cnt + x[u'KAPTDA_CNT']
 #   info.append(cnt)
    return info

In [16]:
apt_info = pd.DataFrame(columns=['KAPT_USEDATE', 'BJD_CODE', 'KAPT_NAME', 'OPEN_TERM', 'KAPT_CODE',
       'OCCU_FIRST_DATE', 'X', 'Y', 'ENERGY_B_COUNT', 'BJD_NAME'])

In [17]:
for code in dong_info.apply(lambda x : x['si_code']*1000000+x['gu_code']*1000+x['dong_code'], axis=1):
    apt = getAptInfo(code)
    pd.concat([apt_info,apt],axis = 1)

In [18]:
codes = dong_info.apply(lambda x : x['si_code']*1000000+x['gu_code']*1000+x['dong_code'], axis=1)
apt1 = getAptInfo(codes[0])
apt2 = getAptInfo(codes[1])
apt3 = getAptInfo(codes[2])
apt4 = getAptInfo(codes[3])
apt5 = getAptInfo(codes[4])
apt6 = getAptInfo(codes[5])
apt7 = getAptInfo(codes[6])
apt8 = getAptInfo(codes[7])
apt9 = getAptInfo(codes[8])
apt10 = getAptInfo(codes[9])
apt11 = getAptInfo(codes[10])
apt12 = getAptInfo(codes[11])
apt13 = getAptInfo(codes[12])
apt14 = getAptInfo(codes[13])
apt_info = pd.concat([apt1,apt2,apt3,apt4,apt5,apt6,apt7,apt8,apt9,apt10,apt11,apt12,apt13,apt14],axis = 0)

In [19]:
apt_info
apt_info.index = range(173)

In [21]:
# crewling apt detail 정보( 세대수, 건립년도, 인근 지하철 등 ) 
apt_details = pd.DataFrame(columns=['KAPT_CODE','CONVENIENT_FACILITY','EDUCATION_FACILITY','KAPTD_PCNT','KAPTD_PCNTU', 'KAPTD_WTIMEBUS', 'KAPTD_WTIMESUB','SUBWAY_LINE','SUBWAY_STATION', 'KAPTDA_CNT1', 'KAPTDA_CNT2', 'KAPTDA_CNT3', 'KAPTDA_CNT4'])

for code in apt_info['KAPT_CODE']:
    a = getAptDetailInfo(code)
    if not bool(a):
        continue
    info = extractAptInfo(a)
    if not bool(info):
        continue
    n=len(apt_details)
    apt_details.loc[n] = info
    print ("%s apt data collection completed(%d)" % (code,n))

A13593901 apt data collection completed(0)
A13593908 apt data collection completed(1)
A13524006 apt data collection completed(2)
A13580602 apt data collection completed(3)
A13524009 apt data collection completed(4)
A13580801 apt data collection completed(5)
A13524004 apt data collection completed(6)
A13580002 apt data collection completed(7)
A13580601 apt data collection completed(8)
A13580204 apt data collection completed(9)
A13580103 apt data collection completed(10)
A13580003 apt data collection completed(11)
A13599402 apt data collection completed(12)
A13599303 apt data collection completed(13)
A13599301 apt data collection completed(14)
A13524005 apt data collection completed(15)
A13580001 apt data collection completed(16)
A13594007 apt data collection completed(17)
A10025203 apt data collection completed(18)
A10025675 apt data collection completed(19)
A13583301 apt data collection completed(20)
A13582002 apt data collection completed(21)
A13501004 apt data collection completed(22

HTTPError: HTTP Error 502: Proxy Error

In [22]:
apt_details

,KAPT_CODE,CONVENIENT_FACILITY,EDUCATION_FACILITY,KAPTD_PCNT,KAPTD_PCNTU,KAPTD_WTIMEBUS,KAPTD_WTIMESUB,SUBWAY_LINE,SUBWAY_STATION,KAPTDA_CNT1,KAPTDA_CNT2,KAPTDA_CNT3,KAPTDA_CNT4
0,A13593901,관공서(수서경찰서) 병원(삼성의료원) 공원(한마음공원),초등학교(대진) 중학교(중동중학교) 고등학교(중동고등학교),19,483,5분이내,5~10분이내,3호선,대청,0,0,84,128
1,A13593908,"관공서(수서경찰서) 병원(삼성의료원) 백화점(롯데백화점,현대백화점) 공원(한마음공원)","초등학교(대진,양천) 중학교(개원) 고등학교(경기여고)",0,0,5분이내,5분이내,중앙선,대모산,1623,0,0,0
2,A13524006,"관공서(개포4동사무소) 병원(삼성의료원,영동세브란스) 백화점(롯데백화점) 공원(달터공원)",초등학교(구룡) 중학교(구룡) 고등학교(개포),500,0,5~10분이내,10~15분이내,3호선,매봉,0,112,230,216
3,A13580602,"관공서(개포치안센터, 개포119안전센터) 병원(연세대학교강남세브란스병원) 백화점(롯...","초등학교(개원,구룡,개일초) 중학교(구룡중) 고등학교(개포고)",1126,0,5분이내,5~10분이내,"3호선, 분당선",구룡,0,3,387,288
4,A13524009,"관공서(일원동 사무소, 수서경찰서)",-,438,73,5분이내,5~10분이내,3호선,-,1753,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,A13509010,병원(차소아과의원) 공원(삼성은행나무공원),"초등학교(삼릉,언북,학동) 중학교(언주) 고등학교(영동)",10,1368,5분이내,5분이내,"7호선, 분당선",강남구청,0,158,415,140
78,A13509006,관공서(강남구청) 병원(김진섭치과) 백화점(현대) 대형상가(LG하이프라자) 공원(삼...,초등학교(삼릉초) 중학교(언주중) 고등학교(경기고),38,259,5분이내,5분이내,"7호선, 신분당선",강남구청,152,152,0,0
79,A13509101,공원(),초등학교() 중학교(),74,56,5분이내,5~10분이내,7호선,-,0,167,0,0
80,A13509003,관공서(주민센터) 병원(정봉준치과) 백화점(현대) 대형상가(코엑스몰) 공원(청담공원...,초등학교(봉은초) 중학교(봉은중) 고등학교(경기고),61,220,5~10분이내,5~10분이내,"7호선, 9호선",청담,123,75,54,0


In [ ]:
apt_details

In [39]:
getAptDetailInfo(apt_info.KAPT_CODE[5])

{'resultMap_kapt': {'APPL_CODE': None,
  'APPL_PRODATE': '',
  'CODE_APT': '아파트',
  'CODE_CLEAN': '자치관리',
  'CODE_CLTIME': '8',
  'CODE_DISINF': '위탁관리',
  'CODE_ECON': '단일계약',
  'CODE_ELEV': '-',
  'CODE_EMAINT': '-',
  'CODE_EMGR': '위탁선임',
  'CODE_FALARM': 'P형',
  'CODE_GARBAGE': '음식물쓰레기종량제',
  'CODE_HALL': '계단식',
  'CODE_HEAT': '개별난방',
  'CODE_MGR': '자치관리',
  'CODE_NET': '무',
  'CODE_SALE': '분양',
  'CODE_SEC': '위탁관리',
  'CODE_STIME': '21',
  'CODE_STR': '철골철근콘크리트구조',
  'CODE_WSUPPLY': '고가수조식',
  'CONVENIENT_FACILITY': '-',
  'DISPOSAL_TYPE': '도포식,분무식,독이식',
  'EDUCATION_FACILITY': '-',
  'EMPTY_HO_CNT': None,
  'KAPTD_CCCNT': '5',
  'KAPTD_CLCNT': '6',
  'KAPTD_DCNT': '12',
  'KAPTD_ECAPA': '350',
  'KAPTD_ECNTB': 0,
  'KAPTD_ECNTC': 0,
  'KAPTD_ECNTD': 0,
  'KAPTD_ECNTE': 0,
  'KAPTD_ECNTM': 0,
  'KAPTD_ECNTP': 0,
  'KAPTD_PCNT': '1160',
  'KAPTD_PCNTU': '0',
  'KAPTD_SCNT': '12',
  'KAPTD_SEC_COM': '(주)대신씨엔에스',
  'KAPTD_WTIMEBUS': '5분이내',
  'KAPTD_WTIMESUB': '-',
  'KAPT_ACC_COMPANY

In [36]:
'CONVENIENT_FACILITY','EDUCATION_FACILITY','KAPTD_PCNT','KAPTD_PCNTU', 'KAPTD_WTIMEBUS', 'KAPTD_WTIMESUB', 'resultMap_kapt_addrList'

('CONVENIENT_FACILITY',
 'EDUCATION_FACILITY',
 'KAPTD_PCNT',
 'KAPTD_PCNTU',
 'KAPTD_WTIMEBUS',
 'KAPTD_WTIMESUB',
 'resultMap_kapt_addrList')

In [30]:
def extractAptInfo2(a):
    info = []
    for name in ['KAPT_CODE', 'ADDR'] :
        info.append(a['resultMap_kapt_addrList'][0][name]);
    cnt = 0
    for x in a[u'resultMap_kapt_areacnt']:
        if( type(x[u'KAPTDA_CNT']) is int ):
            info.append(x[u'KAPTDA_CNT'])
        else :
            info.append(0)
#           cnt = cnt + x[u'KAPTDA_CNT']
 #   info.append(cnt)
    return info

apt_details2 = pd.DataFrame(columns=['KAPT_CODE','ADDR','KAPTDA_CNT1', 'KAPTDA_CNT2', 'KAPTDA_CNT3', 'KAPTDA_CNT4'])

for code in apt_info['KAPT_CODE']:
    a = getAptDetailInfo(code)
    if not bool(a):
        continue
    info = extractAptInfo2(a)
    if not bool(info):
        continue
    n=len(apt_details2)
    apt_details2.loc[n] = info
    print ("%s apt data collection completed(%d)" % (code,n))

A13593901 apt data collection completed(0)
A13593908 apt data collection completed(1)
A13524006 apt data collection completed(2)
A13580602 apt data collection completed(3)
A13524009 apt data collection completed(4)
A13580801 apt data collection completed(5)
A13524004 apt data collection completed(6)
A13580002 apt data collection completed(7)
A13580601 apt data collection completed(8)
A13580204 apt data collection completed(9)
A13580103 apt data collection completed(10)
A13580003 apt data collection completed(11)
A13599402 apt data collection completed(12)
A13599303 apt data collection completed(13)
A13599301 apt data collection completed(14)
A13524005 apt data collection completed(15)
A13580001 apt data collection completed(16)
A13594007 apt data collection completed(17)
A10025203 apt data collection completed(18)
A10025675 apt data collection completed(19)
A13583301 apt data collection completed(20)
A13582002 apt data collection completed(21)
A13501004 apt data collection completed(22

HTTPError: HTTP Error 502: Proxy Error

In [32]:
def extractAptInfo3(a):
    info = []
    for name in ['KAPT_CODE', 'ADDR'] :
        info.append(a['resultMap_kapt_addrList'][1][name]);
    cnt = 0
    for x in a[u'resultMap_kapt_areacnt']:
        if( type(x[u'KAPTDA_CNT']) is int ):
            info.append(x[u'KAPTDA_CNT'])
        else :
            info.append(0)
#           cnt = cnt + x[u'KAPTDA_CNT']
 #   info.append(cnt)
    return info

apt_details3 = pd.DataFrame(columns=['KAPT_CODE','ADDR','KAPTDA_CNT1', 'KAPTDA_CNT2', 'KAPTDA_CNT3', 'KAPTDA_CNT4'])

for code in apt_info['KAPT_CODE']:
    a = getAptDetailInfo(code)
    if not bool(a):
        continue
    info = extractAptInfo3(a)
    if not bool(info):
        continue
    n=len(apt_details3)
    apt_details3.loc[n] = info
    print ("%s apt data collection completed(%d)" % (code,n))

A13593901 apt data collection completed(0)
A13593908 apt data collection completed(1)
A13524006 apt data collection completed(2)
A13580602 apt data collection completed(3)
A13524009 apt data collection completed(4)


IndexError: list index out of range

In [73]:
apt_details

,KAPT_CODE,CONVENIENT_FACILITY,EDUCATION_FACILITY,KAPTD_PCNT,KAPTD_PCNTU,KAPTD_WTIMEBUS,KAPTD_WTIMESUB,SUBWAY_LINE,SUBWAY_STATION,KAPTDA_CNT1,KAPTDA_CNT2,KAPTDA_CNT3,KAPTDA_CNT4
0,A13593901,관공서(수서경찰서) 병원(삼성의료원) 공원(한마음공원),초등학교(대진) 중학교(중동중학교) 고등학교(중동고등학교),19,483,5분이내,5~10분이내,3호선,대청,0,0,84,128
1,A13593908,"관공서(수서경찰서) 병원(삼성의료원) 백화점(롯데백화점,현대백화점) 공원(한마음공원)","초등학교(대진,양천) 중학교(개원) 고등학교(경기여고)",0,0,5분이내,5분이내,중앙선,대모산,1623,0,0,0
2,A13524006,"관공서(개포4동사무소) 병원(삼성의료원,영동세브란스) 백화점(롯데백화점) 공원(달터공원)",초등학교(구룡) 중학교(구룡) 고등학교(개포),500,0,5~10분이내,10~15분이내,3호선,매봉,0,112,230,216
3,A13580602,"관공서(개포치안센터, 개포119안전센터) 병원(연세대학교강남세브란스병원) 백화점(롯...","초등학교(개원,구룡,개일초) 중학교(구룡중) 고등학교(개포고)",1126,0,5분이내,5~10분이내,"3호선, 분당선",구룡,0,3,387,288
4,A13524009,"관공서(일원동 사무소, 수서경찰서)",-,438,73,5분이내,5~10분이내,3호선,-,1753,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,A13510201,"관공서(강남구청,강남세무서, 청담동주민센터) 병원(우리들병원) 공원(청담목련공원)",초등학교(언북초교) 중학교(언주중학교) 고등학교(영동고교),69,137,5분이내,5분이내,"7호선, 분당선","청담역, 강남구청역",56,82,46,0
169,A10025280,-,-,14,129,5분이내,-,7호선,-,0,0,108,0
170,A13510007,-,-,0,905,5~10분이내,5분이내,7호선,-,388,116,204,0
171,A13510102,병원(김수균외과),초등학교(청담초등학교) 중학교(청담중학교) 고등학교(영동고등학교),60,354,15~20분이내,10~15분이내,7호선,-,101,76,140,0


In [77]:
apt_details2

,KAPT_CODE,ADDR,KAPTDA_CNT1,KAPTDA_CNT2,KAPTDA_CNT3,KAPTDA_CNT4
0,A13593901,서울특별시 강남구 개포동 12-2 엘지개포자이,0,0,84,128
1,A13593908,서울특별시 강남구 개포동 12 대치아파트,1623,0,0,0
2,A13524006,서울특별시 강남구 개포동 654 개포2차현대아파트,0,112,230,216
3,A13580602,서울특별시 강남구 개포동 649 경남아파트,0,3,387,288
4,A13524009,"서울특별시 강남구 개포동 12 대치,대청 아파트",1753,0,0,0
...,...,...,...,...,...,...
168,A13510201,서울특별시 강남구 청담동 10 삼환아파트101동,56,82,46,0
169,A10025280,서울특별시 강남구 청담동 130 청담아이파크,0,0,108,0
170,A13510007,서울특별시 강남구 청담동 134-38 청담자이아파트,388,116,204,0
171,A13510102,서울특별시 강남구 청담동 127-31 청담현대3차아파트,101,76,140,0


In [106]:
apt_details3

,KAPT_CODE,ADDR,KAPTDA_CNT1,KAPTDA_CNT2,KAPTDA_CNT3,KAPTDA_CNT4
0,A13593901,서울특별시 강남구 개포로109길 69,0,0,84,128
1,A13593908,서울특별시 강남구 개포로109길 5,1623,0,0,0
2,A13524006,서울특별시 강남구 언주로 107,0,112,230,216
3,A13580602,서울특별시 강남구 언주로 110,0,3,387,288
4,A13524009,서울특별시 강남구 개포로109길 9,1753,0,0,0


In [78]:
apt_details.to_csv("detail1.csv")

In [79]:
apt_details2.to_csv("detail2.csv")